In [1]:
%%capture
!pip install rioxarray "xarray[complete]" geopandas


In [1]:

import pandas as pd
import boto3
import xarray as xr
import os
import zarr
import s3fs
from dask.diagnostics import ProgressBar
from datetime import datetime
import yaml
from yaml import SafeLoader
import logging
from numcodecs import blosc

In [2]:
confPath = "../../../conf/data/configuration.yml"
s3 = s3fs.S3FileSystem()

In [3]:
def convertRadiation(k):
    c = k/1000000
    return c

def readYamlConf(path: str):
    with open(path,'r') as conf:
        confArgs = yaml.load(conf, Loader= SafeLoader)
        #print(confArgs)
    return confArgs


In [4]:
#confArgs

In [4]:
confArgs = readYamlConf(confPath)['agera5ArgumentsSolarFlux']
ageraS3 = s3fs.S3Map(root=confArgs['awsAgera5SolarFluxRawPath'], s3=s3, check=False)

In [6]:
confArgs['awsAgera5SolarFluxRawPath']

's3://climate-action-datalake/zone=raw/source=agera5/variable=solarRadiationFlux.zarr/'

In [5]:
fileToProcess = os.listdir(confArgs['pathforStandardized'])

In [6]:
#%debug
logging.warning("Loading files to append")
ncToLoad = os.listdir(confArgs['pathforStandardized'])


for _nc in sorted(ncToLoad):
    _file = confArgs['pathforStandardized']+_nc
    logging.warning("Opening %s ", _nc)
    nc = xr.open_dataset(_file)
    nc = nc[['time','lat','lon','Solar-Radiation-Flux']]
    logging.warning("Organizing dimensions")
    compressor = zarr.Blosc(cname='lz4', clevel= 1, shuffle=False)
    blosc.set_nthreads(8)
    encoding = {vname: {'compressor': compressor
                        ,'chunks': (1,2000,7200)} for vname in nc.data_vars}
    with ProgressBar():
        nc.to_zarr(ageraS3,  mode='a-', append_dim='time'
                   )
    logging.warning("Insert finished")


logging.warning("Process completed")


In [9]:
compressor = zarr.Blosc(cname='lz4')
encoding = {vname: {'compressor': compressor,'chunks': (1,2000,7200)} for vname in allFiles.data_vars}
